<a href="https://colab.research.google.com/github/irawan555/MachineLearningTasks/blob/main/13thWeekTasks/BAB%203/Irawan_Mardiansyah_1103210174_A_full_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Irawan Mardiansyah

NIM : 1103210174

Kelas : TK4504

Lecture : 13

# **A full training**

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [23]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [24]:
# Mengimpor fungsi load_dataset dari pustaka datasets
from datasets import load_dataset
# Mengimpor kelas AutoTokenizer dan DataCollatorWithPadding dari pustaka Transformers
from transformers import AutoTokenizer, DataCollatorWithPadding

# Memuat dataset GLUE (General Language Understanding Evaluation) dengan tugas MRPC (Microsoft Research Paraphrase Corpus)
raw_datasets = load_dataset("glue", "mrpc")

# Menentukan model yang akan digunakan dengan checkpoint bert-base-uncased
checkpoint = "bert-base-uncased"
# Membuat objek tokenizer otomatis berdasarkan model yang dipilih
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Membuat fungsi untuk melakukan tokenisasi pada dataset
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Melakukan tokenisasi pada dataset menggunakan fungsi yang telah didefinisikan sebelumnya, dengan batch processing
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Membuat objek data_collator untuk menangani padding pada data dengan tokenizer yang telah digunakan
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [25]:
# Menghapus kolom "sentence1", "sentence2", dan "idx" dari dataset tokenized_datasets
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])

# Mengubah nama kolom "label" menjadi "labels" dalam dataset tokenized_datasets
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# Mengatur format dataset tokenized_datasets menjadi "torch"
tokenized_datasets.set_format("torch")

# Mendapatkan nama kolom dari bagian "train" dalam dataset tokenized_datasets
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [26]:
["attention_mask", "input_ids", "labels", "token_type_ids"]

['attention_mask', 'input_ids', 'labels', 'token_type_ids']

In [27]:
# Mengimpor kelas DataLoader dari modul torch.utils.data
from torch.utils.data import DataLoader

# Membuat DataLoader untuk data latih (train)
# tokenized_datasets["train"]: dataset yang sudah di-tokenisasi untuk data latih
# shuffle=True: data akan diacak setiap epoch
# batch_size=8: jumlah sampel yang akan dimuat dalam setiap batch
# collate_fn=data_collator: fungsi untuk menggabungkan sampel-sampel ke dalam batch
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)

# Membuat DataLoader untuk data evaluasi (validasi)
# tokenized_datasets["validation"]: dataset yang sudah di-tokenisasi untuk data evaluasi
# batch_size=8: jumlah sampel yang akan dimuat dalam setiap batch
# collate_fn=data_collator: fungsi untuk menggabungkan sampel-sampel ke dalam batch
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [28]:
# Mengulang setiap batch dalam dataloader pelatihan (train_dataloader)
for batch in train_dataloader:
    # Menghentikan iterasi setelah satu batch (ini mungkin hanya contoh atau debugging)
    break

# Membuat kamus yang berisi bentuk (shape) dari setiap tensor dalam batch
# 'k' adalah kunci (key), mewakili nama tensor, dan 'v' adalah nilai (value), mewakili bentuk tensor
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 74]),
 'token_type_ids': torch.Size([8, 74]),
 'attention_mask': torch.Size([8, 74])}

In [29]:
# Mengimpor kelas yang diperlukan dari pustaka Transformers untuk model klasifikasi urutan otomatis
from transformers import AutoModelForSequenceClassification

# Membuat objek model klasifikasi urutan otomatis dan memuat berat dari titik kontrol yang disebutkan dengan menentukan jumlah label
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
# Melakukan pemrosesan forward pass model dengan batch data yang diberikan
outputs = model(**batch)

# Mencetak nilai loss dari output model serta bentuk logits yang dihasilkan
print(outputs.loss, outputs.logits.shape)

tensor(0.5788, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [31]:
# Mengimpor kelas optimizer AdamW dari pustaka Transformers
from transformers import AdamW

# Membuat objek optimizer AdamW dengan model.parameters() sebagai parameter untuk dioptimalkan
# dan learning rate (lr) sebesar 5e-5
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [32]:
# Mengimpor fungsi get_scheduler dari pustaka Transformers
from transformers import get_scheduler

# Menentukan jumlah epochs yang akan dilatih
num_epochs = 3

# Menghitung total langkah pelatihan berdasarkan jumlah epochs dan panjang dataloader pelatihan
num_training_steps = num_epochs * len(train_dataloader)

# Membuat penjadwalan laju pembelajaran menggunakan metode linear
# dengan menentukan optimizer, jumlah langkah pemanasan, dan total langkah pelatihan
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# Mencetak total langkah pelatihan
print(num_training_steps)

1377


In [33]:
# Mengimpor pustaka PyTorch
import torch

# Memeriksa ketersediaan GPU, jika tersedia, gunakan GPU, jika tidak, gunakan CPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Memindahkan model ke perangkat yang telah ditentukan (GPU atau CPU)
model.to(device)

# Menampilkan perangkat yang digunakan (GPU atau CPU)
device

device(type='cuda')

In [34]:
# Mengimpor kelas tqdm dari pustaka tqdm untuk menampilkan bar kemajuan
from tqdm.auto import tqdm

# Membuat objek progress_bar dengan rentang num_training_steps untuk menampilkan bar kemajuan saat pelatihan model
progress_bar = tqdm(range(num_training_steps))

# Mengatur model dalam mode pelatihan
model.train()

# Melakukan iterasi sebanyak num_epochs untuk melatih model
for epoch in range(num_epochs):
    # Melakukan iterasi untuk setiap batch dalam train_dataloader
    for batch in train_dataloader:
        # Memindahkan batch ke perangkat yang sesuai (misalnya, GPU)
        batch = {k: v.to(device) for k, v in batch.items()}

        # Memperoleh output dari model berdasarkan batch input
        outputs = model(**batch)

        # Mendapatkan nilai loss dari output model
        loss = outputs.loss

        # Menghitung gradien loss
        loss.backward()

        # Mengupdate parameter model berdasarkan gradien
        optimizer.step()

        # Mengupdate learning rate scheduler
        lr_scheduler.step()

        # Mengatur gradien menjadi nol untuk iterasi selanjutnya
        optimizer.zero_grad()

        # Mengupdate bar kemajuan
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

In [35]:
# Mengimpor fungsi evaluate dari modul evaluate
import evaluate

# Memuat metrik evaluasi untuk dataset GLUE dan task MRPC
metric = evaluate.load("glue", "mrpc")

# Menetapkan mode evaluasi pada model
model.eval()

# Iterasi melalui dataloader untuk data evaluasi
for batch in eval_dataloader:
    # Memindahkan batch ke perangkat (device) yang sesuai (misalnya, GPU)
    batch = {k: v.to(device) for k, v in batch.items()}

    # Mematikan perhitungan gradien saat melakukan evaluasi
    with torch.no_grad():
        # Memperoleh output model menggunakan batch input
        outputs = model(**batch)

    # Mendapatkan prediksi dari logits menggunakan argumen maksimum
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    # Menambahkan batch prediksi dan label referensi ke metrik
    metric.add_batch(predictions=predictions, references=batch["labels"])

# Menghitung dan mengembalikan nilai metrik
metric.compute()

{'accuracy': 0.8651960784313726, 'f1': 0.9046793760831888}

In [36]:
# Mengimpor kelas-kelas yang diperlukan dari pustaka Transformers
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

# Menginisialisasi model berdasarkan checkpoint yang telah disediakan
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Menginisialisasi optimizer AdamW dengan learning rate 3e-5
optimizer = AdamW(model.parameters(), lr=3e-5)

# Memilih device untuk training (cuda jika tersedia, jika tidak, cpu)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)  # Memindahkan model ke device yang dipilih

# Mengatur jumlah epoch dan jumlah total langkah training
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)

# Mengatur scheduler learning rate dengan metode linear
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# Menampilkan progress bar menggunakan tqdm
progress_bar = tqdm(range(num_training_steps))

# Memasuki fase training
model.train()
for epoch in range(num_epochs):
    # Looping melalui data dalam dataloader
    for batch in train_dataloader:
        # Memindahkan batch ke device yang dipilih
        batch = {k: v.to(device) for k, v in batch.items()}
        # Melakukan forward pass
        outputs = model(**batch)
        # Menghitung loss
        loss = outputs.loss
        # Melakukan backpropagation
        loss.backward()
        # Melakukan update parameter dengan optimizer
        optimizer.step()
        # Melakukan update learning rate dengan scheduler
        lr_scheduler.step()
        # Mengosongkan gradient
        optimizer.zero_grad()
        # Memperbarui progress bar
        progress_bar.update(1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1377 [00:00<?, ?it/s]

In [37]:
# Mengimpor kelas-kelas yang diperlukan dari pustaka Accelerate dan Transformers
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

# Inisialisasi objek Accelerator
accelerator = Accelerator()

# Mendapatkan model dari checkpoint yang sudah ada untuk klasifikasi urutan (sequence classification)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Mendefinisikan optimizer dengan AdamW dan learning rate 3e-5
optimizer = AdamW(model.parameters(), lr=3e-5)

# Mempersiapkan data training dan evaluasi menggunakan Accelerator
train_dl, eval_dl, model, optimizer = accelerator.prepare(train_dataloader, eval_dataloader, model, optimizer)

# Mendefinisikan jumlah epoch dan jumlah langkah training
num_epochs = 3
num_training_steps = num_epochs * len(train_dl)

# Mendefinisikan scheduler untuk learning rate
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# Inisialisasi progress bar menggunakan tqdm
progress_bar = tqdm(range(num_training_steps))

# Mengatur model dalam mode pelatihan
model.train()

# Melakukan iterasi untuk setiap epoch
for epoch in range(num_epochs):
    # Melakukan iterasi untuk setiap batch dalam data training
    for batch in train_dl:
        # Melakukan forward pass pada model
        outputs = model(**batch)
        # Mendapatkan loss dari output
        loss = outputs.loss
        # Melakukan backpropagation menggunakan Accelerator
        accelerator.backward(loss)
        # Melakukan optimasi berdasarkan gradient
        optimizer.step()
        # Melakukan update terhadap learning rate
        lr_scheduler.step()
        # Mengatur gradient menjadi nol
        optimizer.zero_grad()
        # Mengupdate progress bar
        progress_bar.update(1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1377 [00:00<?, ?it/s]